In [1]:
import lightgbm as lgb
import numpy as np

from art.attacks import ZooAttack
from art.classifiers import LightGBMClassifier
from art.utils import load_mnist

In [3]:
(x_train, y_train), (x_test, y_test), min_pixel_value, max_pixel_value = load_mnist()

In [4]:
x_test = x_test[0:5]
y_test = y_test[0:5]

In [5]:
nb_samples_train = x_train.shape[0]
nb_samples_test = x_test.shape[0]
x_train = x_train.reshape((nb_samples_train, 28 * 28))
x_test = x_test.reshape((nb_samples_test, 28 * 28))

In [6]:
params = {'objective': 'multiclass', 'metric': 'multi_logloss', 'num_class': 10}
train_set = lgb.Dataset(x_train, label=np.argmax(y_train, axis=1))
test_set = lgb.Dataset(x_test, label=np.argmax(y_test, axis=1))

In [7]:
model = lgb.train(params=params, train_set=train_set, num_boost_round=100, valid_sets=[test_set])

[1]	valid_0's multi_logloss: 1.83284
[2]	valid_0's multi_logloss: 1.54971
[3]	valid_0's multi_logloss: 1.38509
[4]	valid_0's multi_logloss: 1.25221
[5]	valid_0's multi_logloss: 1.15026
[6]	valid_0's multi_logloss: 1.03595
[7]	valid_0's multi_logloss: 0.935987
[8]	valid_0's multi_logloss: 0.847067
[9]	valid_0's multi_logloss: 0.771509
[10]	valid_0's multi_logloss: 0.704675
[11]	valid_0's multi_logloss: 0.648543
[12]	valid_0's multi_logloss: 0.592831
[13]	valid_0's multi_logloss: 0.546758
[14]	valid_0's multi_logloss: 0.50427
[15]	valid_0's multi_logloss: 0.468801
[16]	valid_0's multi_logloss: 0.432248
[17]	valid_0's multi_logloss: 0.401243
[18]	valid_0's multi_logloss: 0.368308
[19]	valid_0's multi_logloss: 0.340732
[20]	valid_0's multi_logloss: 0.315382
[21]	valid_0's multi_logloss: 0.293964
[22]	valid_0's multi_logloss: 0.272822
[23]	valid_0's multi_logloss: 0.253437
[24]	valid_0's multi_logloss: 0.234701
[25]	valid_0's multi_logloss: 0.217876
[26]	valid_0's multi_logloss: 0.203204
[2

In [8]:
classifier = LightGBMClassifier(model=model, clip_values=(min_pixel_value, max_pixel_value))

In [9]:
predictions = classifier.predict(x_test)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print('Accuracy on benign test examples: {}%'.format(accuracy * 100))

Accuracy on benign test examples: 100.0%


In [10]:
attack = ZooAttack(classifier=classifier, confidence=0.5, targeted=False, learning_rate=1e-1, max_iter=200,
                   binary_search_steps=100, initial_const=1e-1, abort_early=True, use_resize=False,
                   use_importance=False, nb_parallel=250, batch_size=1, variable_h=0.01)

In [11]:
x_test_adv = attack.generate(x=x_test)

In [12]:
predictions = classifier.predict(x_test_adv)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print('Accuracy on adversarial test examples: {}%'.format(accuracy * 100))

Accuracy on adversarial test examples: 80.0%


In [15]:
import pickle

In [21]:

from pathlib import Path

In [22]:

path = Path('data')

In [24]:
pickle.dump(x_test_adv, open(path/'adv_examples.pkl', 'wb'))

FileNotFoundError: [Errno 2] No such file or directory: 'data/adv_examples.pkl'